# Log 01-10-2023

## TODOs

- [x] check the generated code
    - [x] variable declaration missing
    - [x] parsing error line 239
    - [x] that parsing error with **

## Observations
### Known issues
- [x] indentation in flow control
- [x] suffix all dropped
- [x] special index
- [x] specific index identified as whole set
- [x] alias
- [x] smin, smax not done right
- [x] comment block
- that special suffix (.resued, .etsolve)
- [x] deal with lead and lag (slice the list)

- comment - seems not possible
    - EBNF is context free
    - comment can be within a set definition - that's a bit annoying

### 19: 57 The issue with exponentiation
- the second `*` parsed as comments
- can also check the returned test to help understand what went wrong

### 20:27

resolved bug1 and exponentiation:
- `WS: (/[ \t\f\r]/+) | (/(\n(?!\*))/+)`
- `%ignore _NL`
- `%ignore COMMENT`
- still have issues with the objective (163)

### 20:52 CORRECT WS SETTING
- `COMMENT : /\n\*[^\n]+/`
- `WS: (/[ \t\f\r]/+) | (/(\n(?!\*))/+)`
- `%ignore _NL`
- `%ignore WS`
- `%ignore COMMENT`

### 20:58
infeasibility caused by constraints?

### 20:53 alt method for dealing with comments
run a method before parsing to reserve all the comments, and then
plug them back by recording their lines

## Code

### Test generated code

In [1]:
from gams2pyomo import GamsParser

f = './test/gams_real_problem/tanksize2.gms'
# f = './test/gams_basic/flow_control/if-elseif-else.gms'
# f = './test/gams_basic/flow_control/if-else.gms'
# f = './test/gams_basic/misc/comments.gms'

with open(f, 'r') as in_file:
    gp = GamsParser(in_file)

parse_tree = gp.parse()
# print(parse_tree.pretty())
res = gp.transform()
# print(res)

In [2]:
with open('tmp/test.py', 'w') as file:
    file.write(res)

### Test generated code

In [13]:

from pyomo.environ import *
import math


m = ConcreteModel()
options = {}
aliases = []
"""

Tank sizing problem based on the three product example in
Rebennack et al., Computers and Chemical Engineering, 2011
This model contains three uncertain product demands.
Number of binary complicating variables:
Number of continuous complicating variables:
Number of binary recourse variables: 0
Number of continuous recourse variables: *s
Number of bilinear terms: *s
Number of univariate signomial terms:
Number of complicating constraints:
Number of recourse constraints:

This file is based on tanksize.350 in the gamslib_ml folder of the GAMS
distribution
compared to Barton's formulation
1. We changed some of the variable bounds
2. reformulate the ProductionUpperBound ProductionLowerBound with Glover inequalites
3. change the storage cost CampaignStorageCost
4. put CampaignInvestmentCost in the objective
"""
options['limrow'] = 0
options['limcol'] = 0
options['optca'] = 1e-09
options['optcr'] = 0.001
options['reslim'] = 10000
options['iterlim'] = 1000000000
options['lp'] = 'CPLEX'
options['nlp'] = 'SNOPT'
options['mip'] = 'CPLEX'
options['minlp'] = 'baron'
m.P = Set(initialize=[1, 2, 3], ordered=True, doc='products')
m.N = Set(initialize=[1, 2, 3], ordered=True, doc='event points')
m.H = Set(initialize=[1, 2], ordered=True, doc='number of scenarios')
m.SUBH = Set(initialize=[1, 2], ordered=True, doc='num. realizations per uncertain param')
aliases.append(['subh', 'subh2', 'subh3'])
aliases.append(['p', 'pp'])
m.VariableInvestmentCostFactor = Param(mutable=True, initialize=0.3271, doc='variable part of the tank investment cost')
m.NumDaysInYear = Param(mutable=True, initialize=365, doc='number of days in a year')
m.MinProductionRate = Param(m.P, mutable=True, initialize={1: 15, 2: 15, 3: 7}, doc='lower bound on the production rate in m^3/day')
m.MaxProductionRate = Param(m.P, mutable=True, initialize={1: 50, 2: 50, 3: 50}, doc='upper bound on the production rate in m^3/day')
m.InventoryLowerBound = Param(m.P, mutable=True, initialize={1: 643, 2: 536, 3: 214}, doc='lower bound on inventory in m^3')
m.InventoryUpperBound = Param(m.P, mutable=True, initialize={1: 4018.36, 2: 3348.63, 3: 1339.45}, doc='upper bound on inventory in m^3')
m.InitialInventory = Param(m.P, mutable=True, doc='initial inventory in m^3')
m.ProductionLength_lower = Param(m.P, mutable=True, initialize={1: 1, 2: 1, 3: 1}, doc='lower bound on production length')
m.ProductionLength_upper = Param(m.P, mutable=True, initialize={1: 10, 2: 10, 3: 10}, doc='upper bound on production length')
m.ProductDemand_nominal = Param(m.P, mutable=True, initialize={1: 4190, 2: 3492, 3: 1397}, doc='nominal demand of product in m^3/year')
m.ProductDemand_stdev = Param(m.P, mutable=True, doc='standard deviation of demand of product in m^3/year')
m.ProductDemand = Param(m.P, m.H, mutable=True, doc='demand of product in m^3/year')
m.CampaignSetupTime = Param(m.P, mutable=True, initialize={1: 0.4, 2: 0.2, 3: 0.1}, doc='campaign setup time in days')
m.CampaignVariableCost = Param(m.P, mutable=True, initialize={1: 18.8304, 2: 19.2934, 3: 19.7563}, doc='tank variable cost per ton')
m.CampaignSetupCost = Param(m.P, mutable=True, initialize={1: 10, 2: 20, 3: 30}, doc='campaign setup cost')
m.prob = Param(m.H, mutable=True, doc='probability of each scenario')
m.subprob = Param(m.SUBH, mutable=True, doc='probability for each individual uncertain realization')
for p in m.P:
	m.InitialInventory[p] = 1.1 * m.InventoryLowerBound[p]
m.ProductDemand_stdev[1] = m.ProductDemand_nominal[1] * 0.1
m.ProductDemand_stdev[2] = m.ProductDemand_nominal[2] * 0.1
m.ProductDemand_stdev[3] = m.ProductDemand_nominal[3] * 0.1
for p in m.P:
	for h in m.H:
		m.ProductDemand[p, h] = m.ProductDemand_nominal[p]
if len(m.H) == 1:
	for h in m.H:
		m.prob[h] = 1
elif len(m.H) == len(m.SUBH):
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 == 1:
			m.subprob[subh] = math.erf(-3 + 6 / len(m.SUBH))
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 > 1 and list(m.SUBH).index(subh) + 1 < len(m.SUBH):
			m.subprob[subh] = math.erf(-3 + list(m.SUBH).index(subh) + 1 * 6 / len(m.SUBH)) - math.erf(-3 + list(m.SUBH).index(subh) + 1 - 1 * 6 / len(m.SUBH))
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 == len(m.SUBH):
			m.subprob[subh] = 1 - math.erf(-3 + len(m.SUBH) - 1 * 6 / len(m.SUBH))
	for subh in m.SUBH:
		for h in m.H:
			if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1:
				m.prob[h] = m.subprob[subh]
		for h in m.H:
			if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1:
				m.ProductDemand[1, h] = m.ProductDemand_nominal[1] - 3 * m.ProductDemand_stdev[1] * 1 - 1 / len(m.SUBH) + list(m.SUBH).index(subh) + 1 - 1 * 6 * m.ProductDemand_stdev[1] / len(m.SUBH)
elif len(m.H) == len(m.SUBH) * len(m.SUBH):
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 == 1:
			m.subprob[subh] = math.erf(-3 + 6 / len(m.SUBH))
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 > 1 and list(m.SUBH).index(subh) + 1 < len(m.SUBH):
			m.subprob[subh] = math.erf(-3 + list(m.SUBH).index(subh) + 1 * 6 / len(m.SUBH)) - math.erf(-3 + list(m.SUBH).index(subh) + 1 - 1 * 6 / len(m.SUBH))
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 == len(m.SUBH):
			m.subprob[subh] = 1 - math.erf(-3 + len(m.SUBH) - 1 * 6 / len(m.SUBH))
	for subh in m.SUBH:
		for subh2 in m.SUBH2:
			for h in m.H:
				if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH):
					m.prob[h] = m.subprob[subh] * m.subprob[subh]
			for h in m.H:
				if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH):
					m.ProductDemand[1, h] = m.ProductDemand_nominal[1] - 3 * m.ProductDemand_stdev[1] * 1 - 1 / len(m.SUBH) + list(m.SUBH).index(subh) + 1 - 1 * 6 * m.ProductDemand_stdev[1] / len(m.SUBH)
			for h in m.H:
				if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH):
					m.ProductDemand[2, h] = m.ProductDemand_nominal[2] - 3 * m.ProductDemand_stdev[2] * 1 - 1 / len(m.SUBH2) + list(m.SUBH2).index(subh2) + 1 - 1 * 6 * m.ProductDemand_stdev[2] / len(m.SUBH2)
elif len(m.H) == len(m.SUBH) * len(m.SUBH) * len(m.SUBH):
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 == 1:
			m.subprob[subh] = math.erf(-3 + 6 / len(m.SUBH))
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 > 1 and list(m.SUBH).index(subh) + 1 < len(m.SUBH):
			m.subprob[subh] = math.erf(-3 + list(m.SUBH).index(subh) + 1 * 6 / len(m.SUBH)) - math.erf(-3 + list(m.SUBH).index(subh) + 1 - 1 * 6 / len(m.SUBH))
	for subh in m.SUBH:
		if len(m.SUBH) > 1 and list(m.SUBH).index(subh) + 1 == len(m.SUBH):
			m.subprob[subh] = 1 - math.erf(-3 + len(m.SUBH) - 1 * 6 / len(m.SUBH))
	for subh in m.SUBH:
		for subh2 in m.SUBH2:
			for subh3 in m.SUBH3:
				for h in m.H:
					if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH) + list(m.SUBH3).index(subh3) + 1 - 1 * len(m.SUBH) * len(m.SUBH2):
						m.prob[h] = m.subprob[subh] * m.subprob[subh] * m.subprob[subh]
				for h in m.H:
					if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH) + list(m.SUBH3).index(subh3) + 1 - 1 * len(m.SUBH) * len(m.SUBH2):
						m.ProductDemand[1, h] = m.ProductDemand_nominal[1] - 3 * m.ProductDemand_stdev[1] * 1 - 1 / len(m.SUBH) + list(m.SUBH).index(subh) + 1 - 1 * 6 * m.ProductDemand_stdev[1] / len(m.SUBH)
				for h in m.H:
					if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH) + list(m.SUBH3).index(subh3) + 1 - 1 * len(m.SUBH) * len(m.SUBH2):
						m.ProductDemand[2, h] = m.ProductDemand_nominal[2] - 3 * m.ProductDemand_stdev[2] * 1 - 1 / len(m.SUBH2) + list(m.SUBH2).index(subh2) + 1 - 1 * 6 * m.ProductDemand_stdev[2] / len(m.SUBH2)
				for h in m.H:
					if list(m.H).index(h) + 1 == list(m.SUBH).index(subh) + 1 + list(m.SUBH2).index(subh2) + 1 - 1 * len(m.SUBH) + list(m.SUBH3).index(subh3) + 1 - 1 * len(m.SUBH) * len(m.SUBH2):
						m.ProductDemand[3, h] = m.ProductDemand_nominal[3] - 3 * m.ProductDemand_stdev[3] * 1 - 1 / len(m.SUBH3) + list(m.SUBH3).index(subh3) + 1 - 1 * 6 * m.ProductDemand_stdev[3] / len(m.SUBH3)
else:
	raise ValueError('ERROR in setting the number of scenarios! Try again')
m.DemandPerDay = Param(m.P, m.H, mutable=True, doc='demand/day/product [tons/day]')
m.TotalDemandPerDay = Param(m.H, mutable=True, doc='total demand/day [tons/day]')
for p in m.P:
	for h in m.H:
		m.DemandPerDay[p, h] = m.ProductDemand[p, h] / m.NumDaysInYear
for h in m.H:
	m.TotalDemandPerDay[h] = sum(m.DemandPerDay[p, h] for p in m.P)
for p in m.P:
	m.CampaignVariableCost[p] = m.CampaignVariableCost[p] / m.NumDaysInYear
m.campaignDuration = Var(m.N, m.H, within=NonNegativeReals, doc='duration of the campaigns')
m.amtProductInCampaign = Var(m.P, m.N, m.H, within=NonNegativeReals, doc='amount of product p produced in campaign n')
m.productInventory = Var(m.P, m.N, m.H, within=NonNegativeReals, doc='amount of product p stored at the beginning of campaign n')
m.productTankSize = Var(m.P, within=NonNegativeReals, doc='size of the product tanks in tons')
m.auxiliaryVariable = Var(m.P, m.N, m.H, within=NonNegativeReals, doc='auxiliary variables')
m.investmentCost = Var(m.H, within=NonNegativeReals, doc='investment costs')
m.setupCost = Var(m.H, within=NonNegativeReals, doc='campaign setup costs')
m.variableCost = Var(m.H, within=NonNegativeReals, doc='variable storage costs')
m.cycleTime = Var(m.H, within=NonNegativeReals, doc='cycle time')
m.costPerTon = Var(m.H, within=NonNegativeReals, doc='cost per ton')
m.campaignLength = Var(m.N, m.H, within=NonNegativeReals, doc='campaignLength is campaignDuration + setuptime ')
m.assignProductToCampaign = Var(m.P, m.N, m.H, within=Binary, doc='binary variable mapping product to campaign')
m.objvar = Var(doc='objective function')
def objective(m):
	return m.objvar == sum(m.prob[h] * m.costPerTon[h] for h in m.H) + m.VariableInvestmentCostFactor * sum(m.prob[h] / m.TotalDemandPerDay[h] for h in m.H) * sum((m.productTankSize[p]) ** 0.5 for p in m.P)
m.objective = Constraint(rule=objective)
def TimeCapacity(m, h):
	return m.cycleTime[h] == sum(m.campaignDuration[n, h] + sum(m.CampaignSetupTime[p] * m.assignProductToCampaign[p, n, h] for p in m.P) for n in m.N)
m.TimeCapacity = Constraint(m.H, rule=TimeCapacity)
def UniqueProduct(m, n, h):
	return sum(m.assignProductToCampaign[p, n, h] for p in m.P) <= 1
m.UniqueProduct = Constraint(m.N, m.H, rule=UniqueProduct)
def MaterialBalance(m, p, n, h):
	return m.productInventory[p, m.N.nextw(n, 1), h] == m.productInventory[p, n, h] + m.amtProductInCampaign[p, n, h] - m.DemandPerDay[p, h] * m.campaignDuration[n, h] + sum(m.CampaignSetupTime[p] * m.assignProductToCampaign[p, n, h] for p in m.P)
m.MaterialBalance = Constraint(m.P, m.N, m.H, rule=MaterialBalance)
def TankCapacity(m, p, n, h):
	return m.productInventory[p, n, h] <= m.productTankSize[p]
m.TankCapacity = Constraint(m.P, m.N, m.H, rule=TankCapacity)
def ProductionUpperBound1(m, p, n, h):
	return m.amtProductInCampaign[p, n, h] <= m.MaxProductionRate[p] * m.ProductionLength_upper[p] * m.assignProductToCampaign[p, n, h]
m.ProductionUpperBound1 = Constraint(m.P, m.N, m.H, rule=ProductionUpperBound1)
def ProductionLowerBound1(m, p, n, h):
	return m.amtProductInCampaign[p, n, h] == m.MinProductionRate[p] * m.ProductionLength_lower[p] * m.assignProductToCampaign[p, n, h]
m.ProductionLowerBound1 = Constraint(m.P, m.N, m.H, rule=ProductionLowerBound1)
def ProductionUpperBound2(m, p, n, h):
	return m.amtProductInCampaign[p, n, h] <= m.MaxProductionRate[p] * m.campaignDuration[n, h]
m.ProductionUpperBound2 = Constraint(m.P, m.N, m.H, rule=ProductionUpperBound2)
def ProductionLowerBound2(m, p, n, h):
	return m.amtProductInCampaign[p, n, h] == m.MinProductionRate[p] * m.campaignDuration[n, h] - m.ProductionLength_upper[p] * 1 - m.assignProductToCampaign[p, n, h]
m.ProductionLowerBound2 = Constraint(m.P, m.N, m.H, rule=ProductionLowerBound2)
def CampaignUpperBound(m, n, h):
	return m.campaignDuration[n, h] <= sum(m.ProductionLength_upper[p] * m.assignProductToCampaign[p, n, h] for p in m.P)
m.CampaignUpperBound = Constraint(m.N, m.H, rule=CampaignUpperBound)
def CampaignLowerBound(m, n, h):
	return m.campaignDuration[n, h] == sum(m.ProductionLength_lower[p] * m.assignProductToCampaign[p, n, h] for p in m.P)
m.CampaignLowerBound = Constraint(m.N, m.H, rule=CampaignLowerBound)
def CampanLengthDef(m, n, h):
	return m.campaignLength[n, h] == m.campaignDuration[n, h] + sum(m.CampaignSetupTime[p] * m.assignProductToCampaign[p, n, h] for p in m.P)
m.CampanLengthDef = Constraint(m.N, m.H, rule=CampanLengthDef)
def CampaignSetupCostCon(m, h):
	return m.setupCost[h] == sum(m.CampaignSetupCost[p] * m.assignProductToCampaign[p, n, h] for p in m.P for n in m.N)
m.CampaignSetupCostCon = Constraint(m.H, rule=CampaignSetupCostCon)
def CampaignStorageCost(m, h):
	return m.variableCost[h] == sum(m.CampaignVariableCost[p] * m.auxiliaryVariable[p, n, h] * m.campaignLength[n, h] for p in m.P for n in m.N)
m.CampaignStorageCost = Constraint(m.H, rule=CampaignStorageCost)
def AuxiliaryCon(m, p, n, h):
	return m.auxiliaryVariable[p, n, h] == 0.5 * m.productInventory[p, m.N.nextw(n, 1), h] + m.productInventory[p, n, h] - m.InventoryLowerBound[p]
m.AuxiliaryCon = Constraint(m.P, m.N, m.H, rule=AuxiliaryCon)
def CampaignCostPerTon(m, h):
	return m.costPerTon[h] * m.cycleTime[h] * m.TotalDemandPerDay[h] == m.setupCost[h] + m.variableCost[h]
m.CampaignCostPerTon = Constraint(m.H, rule=CampaignCostPerTon)
def Sequence(m, p, n, h):
	return 1 - m.assignProductToCampaign[p, n, h] == m.assignProductToCampaign[p, m.N.next(n, 1), h]
m.Sequence = Constraint(m.P, list(m.N)[:-1], m.H, rule=Sequence)
def BreakSymmetry(m, n, h):
	return sum(m.assignProductToCampaign[p, n, h] for p in m.P) == sum(m.assignProductToCampaign[p, m.N.next(n, 1), h] for p in m.P)
m.BreakSymmetry = Constraint(list(m.N)[:-1], m.H, rule=BreakSymmetry)
for n in m.N:
	for h in m.H:
		m.campaignDuration[n, h].setub(max([m.ProductionLength_upper[p].value for p in m.P]))
for p in m.P:
	for n in m.N:
		for h in m.H:
			m.amtProductInCampaign[p, n, h].setub(m.MaxProductionRate[p] * m.ProductionLength_upper[p])
for p in m.P:
	for n in m.N:
		for h in m.H:
			m.productInventory[p, n, h].setlb(m.InventoryLowerBound[p])
for p in m.P:
	for n in m.N:
		for h in m.H:
			m.productInventory[p, n, h].setub(m.InventoryUpperBound[p])
for h in m.H:
	m.productInventory[1, 1, h].fix(m.InventoryLowerBound[1])
for p in m.P:
	for n in m.N:
		for h in m.H:
			m.auxiliaryVariable[p, n, h].setub(m.InventoryUpperBound[p] - m.InventoryLowerBound[p] / 30)
for h in m.H:
	m.cycleTime[h].setub(len(m.N) * max([m.ProductionLength_upper[p].value for p in m.P]) + sum(m.CampaignSetupTime[p] for p in m.P))
for p in m.P:
	for h in m.H:
		m.assignProductToCampaign[p, 1, h].fix(0)
for h in m.H:
	m.assignProductToCampaign[1, 1, h].fix(1)
for h in m.H:
	m.assignProductToCampaign[1, 2, h].fix(0)
for p in m.P:
	m.productTankSize[p].setlb(m.InventoryLowerBound[p])
for p in m.P:
	m.productTankSize[p].setub(m.InventoryUpperBound[p])
for n in m.N:
	for h in m.H:
		m.campaignLength[n, h].setlb(min([m.ProductionLength_lower[p].value + m.CampaignSetupTime[p].value for p in m.P]))
for n in m.N:
	for h in m.H:
		m.campaignLength[n, h].setub(max([m.ProductionLength_upper[p].value + m.CampaignSetupTime[p].value for p in m.P]))
m._obj_ = Objective(rule=m.objvar, sense=1)
opt = SolverFactory(options['minlp'])
opt.solve(m)

{'Problem': [{'Name': 'problem', 'Lower bound': -1e+51, 'Upper bound': 1e+51, 'Number of objectives': 1, 'Number of constraints': 176, 'Number of variables': 97, 'Sense': 'unknown', 'Missing bounds': '0', 'Iterations': '-1', 'Node opt': '-3', 'Node memmax': '0', 'Cpu time': 0.02, 'Wall time': 0.03}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Error rc': 0, 'Time': 0.6691169738769531}]}

In [15]:
m.subprob.pprint()

subprob : probability for each individual uncertain realization
    Size=2, Index=SUBH, Domain=Any, Default=None, Mutable=True
    Key : Value
      1 :               0.0
      2 : 1.999999984582742


### parse bugs

In [1]:
from gams2pyomo import GamsParser

# f = './test/known_bugs/exponentiation.gms'
# f = './test/known_bugs/bug1.gms'
f = './test/gams_real_problem/tanksize2.gms'
# f = './test/gams_basic/misc/comments.gms'

with open(f, 'r') as in_file:
    gp = GamsParser(in_file)

parse_tree = gp.parse()
# print(parse_tree.pretty())
# print(parse_tree.pretty())
# print(parse_tree.pretty())
# print(parse_tree.pretty())
res = gp.transform()
print(res)


from pyomo.environ import *
import math


m = ConcreteModel()
options = {}
aliases = []
"""

Tank sizing problem based on the three product example in
Rebennack et al., Computers and Chemical Engineering, 2011
This model contains three uncertain product demands.
Number of binary complicating variables:
Number of continuous complicating variables:
Number of binary recourse variables: 0
Number of continuous recourse variables: *s
Number of bilinear terms: *s
Number of univariate signomial terms:
Number of complicating constraints:
Number of recourse constraints:

This file is based on tanksize.350 in the gamslib_ml folder of the GAMS
distribution
compared to Barton's formulation
1. We changed some of the variable bounds
2. reformulate the ProductionUpperBound ProductionLowerBound with Glover inequalites
3. change the storage cost CampaignStorageCost
4. put CampaignInvestmentCost in the objective
"""
options['limrow'] = 0
options['limcol'] = 0
options['optca'] = 1e-09
options['optcr'] = 